In [1]:
# 1.Load data
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [29]:
# 2. Define Net
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # my first CNN Net   ( conv  - relu - maxpool ) * 3 + fc * 3
    # input (4,3,32,32)
    # output (4,10)
    def __init__(self):
        super(Net,self).__init__()
        
        # (3,32,32) -> (6,16,16)
        layer1 = nn.Sequential()
        layer1.add_module("conv1",nn.Conv2d(3,6,3,stride=1,padding=1))
        layer1.add_module("relu1",nn.ReLU(True))
        layer1.add_module("pool1",nn.MaxPool2d(2,stride=2))
        self.layer1 = layer1
        
        # (6,16,16) -> (12,8,8)
        layer2 = nn.Sequential()
        layer2.add_module("conv2",nn.Conv2d(6,12,3,stride=1,padding=1))
        layer2.add_module("relu2",nn.ReLU(True))
        layer2.add_module("pool2",nn.MaxPool2d(2,stride=2))
        self.layer2 = layer2
        
        # (12,8,8) -> (24,4,4)
        layer3 = nn.Sequential()
        layer3.add_module("conv3",nn.Conv2d(12,24,3,stride=1,padding=1))
        layer3.add_module("relu3",nn.ReLU(True))
        layer3.add_module("pool3",nn.MaxPool2d(2,stride=2))
        self.layer3 = layer3
        
        layer4 = nn.Sequential()
        layer4.add_module("fc1",nn.Linear(24*4*4, 120))
        layer4.add_module("relu4",nn.ReLU(True))
        layer4.add_module("fc2",nn.Linear(120, 84))
        layer4.add_module("relu5",nn.ReLU(True))
        layer4.add_module("fc3",nn.Linear(84, 10))
        self.layer4 = layer4
    
    def forward(self, x):
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        conv3 = self.layer3(conv2)
        mid = conv3.view(conv3.size(0),-1)
        #print(mid.size())
        out = self.layer4(mid)
        return out
    
net = Net()
print(net)
        
        

Net(
  (layer1): Sequential(
    (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (fc1): Linear(in_features=384, out_features=120, bias=True)
    (relu4): ReLU(inplace)
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (relu5): ReLU(inplace)
    (fc3): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [30]:
# 3.Define loss function and update strategy
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9)

In [31]:
# 4.Train the net
epoch = 2
for i in range(epoch):
    for j, data in enumerate(trainloader):
        imageBatch, labels = data
        #print(imageBatch.size())
        optimizer.zero_grad()
        output = net.forward(imageBatch)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        if j%2000 == 0:
            print("epoch:%2d  iteration:%2d ----  loss:%2f"%(i,j,loss))
print("finish")

epoch: 0  iteration: 0 ----  loss:2.295497
epoch: 0  iteration:2000 ----  loss:2.333959
epoch: 0  iteration:4000 ----  loss:2.450923
epoch: 0  iteration:6000 ----  loss:1.757332
epoch: 0  iteration:8000 ----  loss:1.256238
epoch: 0  iteration:10000 ----  loss:2.483727
epoch: 0  iteration:12000 ----  loss:1.194417
epoch: 1  iteration: 0 ----  loss:1.355987
epoch: 1  iteration:2000 ----  loss:1.868031
epoch: 1  iteration:4000 ----  loss:1.430183
epoch: 1  iteration:6000 ----  loss:1.269344
epoch: 1  iteration:8000 ----  loss:0.803906
epoch: 1  iteration:10000 ----  loss:1.988281
epoch: 1  iteration:12000 ----  loss:0.791740
finish


In [37]:
# 5.Evaluate the net
total = 0
correct = 0
for data in testloader:
    imageBatch, labels = data
    output = net.forward(imageBatch)
    _,predict = torch.max(output,1)
    #print(labels.size())
    #print(predict.size())
    total += labels.size(0)
    correct += (predict == labels).sum()
    #print(correct.size())
    


correct:5335
total:10000
accuracy:0.000000


In [43]:
print("correct:%d"%(correct))
print("total:%d"%(total))
print("accuracy:%.2f%%"%(float(correct)/ total*100))

correct:5335
total:10000
accuracy:53.35%
